In [2]:
import numpy as np
import pandas as pd
import math
import scipy

# Collect data

### demographics

In [3]:
#user= dempgraphics?
users=pd.read_csv('demographics.csv')
users.columns=['Customer ID','Count','Gender','Age','Under_30','Senior_Citizen','Married','Dependents','Number_of_Dependents']
# do we still need Under 30/ Senior Citizen, or we need to group the age ?
users.drop(['Count'],axis=1,inplace=True)
users.drop(['Under_30','Senior_Citizen','Dependents'],axis=1,inplace=True)
users

,Customer ID,Gender,Age,Married,Number_of_Dependents
0,1746-TGTWV,Male,78.0,No,0.0
1,8499-BRXTD,Female,74.0,Yes,1.0
2,2041-JIJCI,Male,71.0,No,3.0
3,5832-EXGTT,Female,78.0,Yes,1.0
4,8350-NYMVI,NaN,80.0,NaN,1.0
...,...,...,...,...,...
6158,7609-NRNCA,Female,NaN,No,0.0
6159,4835-YSJMR,Male,38.0,NaN,2.0
6160,5067-DGXLL,NaN,30.0,Yes,2.0
6161,3129-AAQOU,Female,32.0,Yes,2.0


### location

In [4]:
location=pd.read_csv('location.csv')
location.columns=['Customer ID','Count','Country','State','City','Zip Code','Lat Long','Latitude','Longtitude']
# Country, State are the same, latitude/longtitude's information is in Zip code
location.drop(['Count','Country','State','Lat Long','Latitude','Longtitude'],axis=1,inplace=True)
#location.drop(['City'],axis=1,inplace=True)

### population

In [5]:
population=pd.read_csv('population.csv')
population.columns=['ID','Zip Code','Population']
population.drop(['ID'],axis=1,inplace=True)
population

,Zip Code,Population
0,90001,54492
1,90002,44586
2,90003,58198
3,90004,67852
4,90005,43019
...,...,...
1666,96145,4002
1667,96146,942
1668,96148,678
1669,96150,33038


### merge location and population ,and join users

In [6]:
location=pd.merge(location,population,on='Zip Code')
users=pd.merge(users,location,on='Customer ID',how='outer')
users

,Customer ID,Gender,Age,Married,Number_of_Dependents,City,Zip Code,Population
0,1746-TGTWV,Male,78.0,No,0.0,Los Angeles,90022.0,68701.0
1,8499-BRXTD,Female,74.0,Yes,1.0,NaN,NaN,NaN
2,2041-JIJCI,Male,71.0,No,3.0,NaN,NaN,NaN
3,5832-EXGTT,Female,78.0,Yes,1.0,Inglewood,90303.0,27778.0
4,8350-NYMVI,NaN,80.0,NaN,1.0,Whittier,90602.0,26265.0
...,...,...,...,...,...,...,...,...
6853,6839-ITVZJ,NaN,NaN,NaN,NaN,Jacumba,91934.0,699.0
6854,6821-JPCDC,NaN,NaN,NaN,NaN,Glennville,93226.0,296.0
6855,5799-JRCZO,NaN,NaN,NaN,NaN,Bakersfield,93311.0,20440.0
6856,8741-LQOBK,NaN,NaN,NaN,NaN,Mccloud,96057.0,1586.0


satisfaction

In [7]:
satisfaction=pd.read_csv('satisfaction.csv')
satisfaction.columns=['Customer ID','score']
users=pd.merge(users,satisfaction,on='Customer ID',how='outer')

services

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
services=pd.read_csv('services.csv')
services.columns=['Customer ID','Count','Quarter','Referred_a_friend','Number of Referrals','Tenure in Months','Offer','Phone_Service','Avg Monthly Long Distance Charges','Multiple_Lines','Internet_Service','Internet_Type','Avg Monthly GB Download','Online_Security','Online_Backup','Device_Protection_Plan','Premium_Tech_Support','Streaming_TV','Streaming_Movies','Streaming_Music','Unlimited_Data','Contract','Paperless_Billing','Payment_Method','Monthly Charge','Total Charges','Total Refunds','Total Extra Data Charges','Total Long Distance Charges','Total Revenue']
# Quarter are always Q3
services.drop(['Count','Quarter'],axis=1,inplace=True)
users=pd.merge(users,services,on='Customer ID',how='outer')
#columns_to_encode = [1,3,4,5,6,8,12,13,14,15,16,17,18,20,21,22,23,34,35,36,27,28,29]
#ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), columns_to_encode)], remainder='passthrough')

# give null a category?
#users.loc[users.Married.isnull(),'Married']='None'
users = pd.concat((users,pd.get_dummies(users.Married,prefix='Married')),1)
users = pd.concat((users,pd.get_dummies(users.Gender,prefix='Gender')),1)
users["City"]=pd.util.hash_array(users["City"].to_numpy())
#users = pd.concat((users,pd.get_dummies(users.Under_30,prefix='Under_30')),1)
#users = pd.concat((users,pd.get_dummies(users.Senior_Citizen,prefix='Senior_Citizen')),1)
#users = pd.concat((users,pd.get_dummies(users.Dependents,prefix='Dependents')),1)
#users = pd.concat((users,pd.get_dummies(users.City,prefix='City')),1)
users = pd.concat((users,pd.get_dummies(users.Referred_a_friend,prefix='Referred_a_friend')),1)
users = pd.concat((users,pd.get_dummies(users.Offer,prefix='Offer')),1)
users = pd.concat((users,pd.get_dummies(users.Phone_Service,prefix='Phone_Service')),1)
users = pd.concat((users,pd.get_dummies(users.Multiple_Lines,prefix='Multiple_Lines')),1)
users = pd.concat((users,pd.get_dummies(users.Internet_Service,prefix='Internet_Service')),1)
users = pd.concat((users,pd.get_dummies(users.Internet_Type,prefix='Internet_Type')),1)
users = pd.concat((users,pd.get_dummies(users.Online_Security,prefix='Online_Security')),1)
users = pd.concat((users,pd.get_dummies(users.Online_Backup,prefix='Online_Backup')),1)
users = pd.concat((users,pd.get_dummies(users.Device_Protection_Plan,prefix='Device_Protection_Plan')),1)
users = pd.concat((users,pd.get_dummies(users.Premium_Tech_Support,prefix='Premium_Tech_Support')),1)
users = pd.concat((users,pd.get_dummies(users.Streaming_TV,prefix='Streaming_TV')),1)
users = pd.concat((users,pd.get_dummies(users.Streaming_Movies,prefix='Streaming_Movies')),1)
users = pd.concat((users,pd.get_dummies(users.Streaming_Music,prefix='Streaming_Music')),1)
users = pd.concat((users,pd.get_dummies(users.Unlimited_Data,prefix='Unlimited_Data')),1)
users = pd.concat((users,pd.get_dummies(users.Contract,prefix='Contract')),1)
users = pd.concat((users,pd.get_dummies(users.Paperless_Billing,prefix='Paperless_Billing')),1)
users = pd.concat((users,pd.get_dummies(users.Payment_Method,prefix='Payment_Method')),1)

users.drop(['Married','Gender','Phone_Service','Multiple_Lines','Internet_Service','Referred_a_friend','Offer','Internet_Type','Online_Security','Online_Backup','Device_Protection_Plan','Premium_Tech_Support','Streaming_TV','Streaming_Movies','Streaming_Music','Unlimited_Data','Contract','Paperless_Billing','Payment_Method'],axis=1,inplace=True)

users

/var/folders/sm/dnfqlxvd0d92nydq65rfrf980000gn/T/ipykernel_2731/2579746403.py:13: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  users = pd.concat((users,pd.get_dummies(users.Married,prefix='Married')),1)
/var/folders/sm/dnfqlxvd0d92nydq65rfrf980000gn/T/ipykernel_2731/2579746403.py:14: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  users = pd.concat((users,pd.get_dummies(users.Gender,prefix='Gender')),1)
/var/folders/sm/dnfqlxvd0d92nydq65rfrf980000gn/T/ipykernel_2731/2579746403.py:20: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  users = pd.concat((users,pd.get_dummies(users.Referred_a_friend,prefix='Referred_a_friend')),1)
/var/folders/sm/dnfqlxvd0d92nydq65rfrf980000gn/T/ipykernel_2731/2579746403.py:21: FutureWarning: In a future version of pandas all argument

,Customer ID,Age,Number_of_Dependents,City,Zip Code,Population,score,Number of Referrals,Tenure in Months,Avg Monthly Long Distance Charges,...,Unlimited_Data_No,Unlimited_Data_Yes,Contract_Month-to-Month,Contract_One Year,Contract_Two Year,Paperless_Billing_No,Paperless_Billing_Yes,Payment_Method_Bank Withdrawal,Payment_Method_Credit Card,Payment_Method_Mailed Check
0,1746-TGTWV,78.0,0.0,14348480317110559103,90022.0,68701.0,NaN,0.0,1.0,0.00,...,1,0,1,0,0,0,1,1,0,0
1,8499-BRXTD,74.0,1.0,18446744073709551615,NaN,NaN,3.0,1.0,8.0,48.85,...,0,1,1,0,0,0,1,0,0,0
2,2041-JIJCI,71.0,3.0,18446744073709551615,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,5832-EXGTT,78.0,1.0,16801335329401301863,90303.0,27778.0,2.0,1.0,25.0,19.76,...,0,1,1,0,0,0,1,1,0,0
4,8350-NYMVI,80.0,1.0,4904908573160186164,90602.0,26265.0,2.0,1.0,37.0,6.33,...,0,1,1,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7037,0516-WJVXC,NaN,NaN,18446744073709551615,NaN,NaN,NaN,2.0,1.0,34.28,...,0,0,1,0,0,1,0,0,1,0
7038,0388-EOPEX,NaN,NaN,18446744073709551615,NaN,NaN,NaN,0.0,35.0,35.50,...,1,0,0,1,0,1,0,0,1,0
7039,7055-JCGNI,NaN,NaN,18446744073709551615,NaN,NaN,NaN,0.0,NaN,27.75,...,0,1,0,0,1,1,0,0,1,0
7040,6891-JPYFF,NaN,NaN,18446744073709551615,NaN,NaN,NaN,0.0,1.0,38.05,...,0,1,1,0,0,0,1,0,1,0


status

In [9]:
status=pd.read_csv('status.csv')
status.columns=['Customer ID','Churn Category']
status['ans'] = status['Churn Category']
status.loc[status.ans=='No Churn','ans']='0'
status.loc[status.ans=='Competitor','ans']='1'
status.loc[status.ans=='Dissatisfaction','ans']='2'
status.loc[status.ans=='Attitude','ans']='3'
status.loc[status.ans=='Price','ans']='4'
status.loc[status.ans=='Other','ans']='5'
status.drop(['Churn Category'],axis=1,inplace=True)
status.rename(columns={'ans':'Churn Category'}, inplace=True)
#final train data
train=pd.merge(status,users,on='Customer ID',how='left')

# Train Model

In [10]:
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SVMSMOTE
from libsvm.svmutil import *

In [11]:
features=list(train)
#print(features)
features.remove('Customer ID')
features.remove('Churn Category')
#features.remove('Referred_a_Friend')
#features.remove('Offer')
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
imputer = imputer.fit(train.loc[:, features])
train_imputed = imputer.fit_transform(train.loc[:, features])
df=pd.DataFrame(train_imputed)
poly = PolynomialFeatures(degree=2, interaction_only=False)
train_imputed = pd.DataFrame(poly.fit_transform(df))
scalar=StandardScaler()
train_imputed=scalar.fit_transform(train_imputed)
train_imputed = pd.DataFrame(train_imputed)
train_imputed


,0,1,2,3,4,5,6,7,8,9,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,0.0,-0.651541,-0.411472,1.022156,0.025953,-0.191061,-2.105970,-0.526417,0.156651,-0.022530,...,1.125145,1.794931,-0.391241,-0.128835,1.177952,0.0,0.0,-0.657680,0.0,-0.208463
1,0.0,0.502076,-0.411472,1.020401,0.053072,-0.350914,-2.105970,-0.526417,-1.418566,-1.705056,...,-0.888774,-0.557124,-0.391241,-0.128835,-0.848931,0.0,0.0,-0.657680,0.0,-0.208463
2,0.0,-0.040803,-0.411472,-0.524565,-0.508422,0.570800,-0.190587,0.961885,0.156651,1.347453,...,-0.888774,-0.557124,-0.391241,-0.128835,1.177952,0.0,0.0,-0.657680,0.0,-0.208463
3,0.0,-1.601578,-0.411472,-1.811784,1.020123,0.850185,-0.190587,-0.526417,-1.140587,-0.099177,...,-0.888774,-0.557124,-0.391241,-0.128835,-0.848931,0.0,0.0,-0.657680,0.0,-0.208463
4,0.0,-0.380102,-0.411472,-1.544837,1.633390,-1.125985,-0.190587,2.450186,1.685539,-0.431069,...,-0.888774,-0.557124,-0.391241,-0.128835,-0.848931,0.0,0.0,1.520497,0.0,-0.208463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4221,0.0,-1.601578,-0.411472,1.022156,0.025953,-0.191061,0.767105,-0.526417,-1.140587,-1.705056,...,1.125145,-0.557124,2.555969,-0.128835,-0.848931,0.0,0.0,1.520497,0.0,-0.208463
4222,0.0,-0.040803,-0.411472,-0.100183,0.278656,-0.664359,-0.190587,-0.526417,-0.260318,1.695716,...,-0.888774,-0.557124,-0.391241,-0.128835,-0.848931,0.0,0.0,1.520497,0.0,-0.208463
4223,0.0,0.909235,1.941546,1.022156,0.025953,-0.191061,0.767105,-0.526417,-0.491968,-0.248752,...,-0.888774,-0.557124,-0.391241,-0.128835,-0.848931,0.0,0.0,1.520497,0.0,-0.208463
4224,0.0,-0.040803,3.118056,-1.811784,1.017658,1.307248,-0.190587,-0.526417,-0.723617,0.321268,...,1.125145,-0.557124,2.555969,-0.128835,-0.848931,0.0,0.0,1.520497,0.0,-0.208463


In [12]:
# 10% vaildation 
x_test,x_train,y_test,y_train = train_test_split(train_imputed,train.loc[:, 'Churn Category'],test_size=0.90, random_state=0)

In [13]:
ros = RandomOverSampler(random_state=0)
#x_train, y_train = ros.fit_resample(x_train, y_train)
#x_train, y_train = BorderlineSMOTE().fit_resample(x_train, y_train)
#x_train, y_train = SVMSMOTE().fit_resample(x_train, y_train)

### SVM 

In [16]:

y_train_num=y_train.to_numpy()
y_train_num=y_train_num.astype(np.int)
x_train_num=x_train.to_numpy()
prob=svm_problem(y_train_num,x_train_num)
param = svm_parameter('-t 0 -c 0.5 -q')
libsvm_train=svm_train(prob,param)
y_test_num=y_test.to_numpy()
y_test_num=y_test_num.astype(np.int)
x_test_num=x_test.to_numpy()
p_label, p_acc, p_val=svm_predict(y_train_num,x_train_num,libsvm_train)

Accuracy = 98.1073% (3732/3804) (classification)


In [17]:
y_test_num=y_test_num.astype(np.int)
p_label, p_acc, p_val=svm_predict(y_test_num,x_test_num,libsvm_train)
score=f1_score(y_test_num,p_label,average='macro')
print(score)
matrix=confusion_matrix(y_test_num,p_label)
print(matrix)
print(classification_report(y_test_num, p_label))
y_train_all=train.loc[:,'Churn Category'].to_numpy()
y_train_all=y_train_all.astype(np.int)
x_train_all=train_imputed.to_numpy()
prob_all=svm_problem(y_train_all,x_train_all)
libsvm_train=svm_train(prob_all,param)

Accuracy = 78.436% (331/422) (classification)
0.30490277421993595
[[298   7   0   1   0   1]
 [ 15  28   0   7   2   1]
 [  8   5   2   4   1   0]
 [  9   6   1   2   1   0]
 [  6   4   0   2   1   0]
 [  2   4   0   2   2   0]]
              precision    recall  f1-score   support

           0       0.88      0.97      0.92       307
           1       0.52      0.53      0.52        53
           2       0.67      0.10      0.17        20
           3       0.11      0.11      0.11        19
           4       0.14      0.08      0.10        13
           5       0.00      0.00      0.00        10

    accuracy                           0.78       422
   macro avg       0.39      0.30      0.30       422
weighted avg       0.75      0.78      0.75       422



### decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtree_model = DecisionTreeClassifier(max_depth = 1).fit(x_train, y_train)
dtree_predictions = dtree_model.predict(x_test)
matrix=confusion_matrix(y_test,dtree_predictions)
print(matrix)
score=f1_score(y_test,dtree_predictions,average='macro')
print(score)
#accuracy = dtree_model.score(x_test, y_test)
#print("dtree",accuracy)
dtree_model = DecisionTreeClassifier(max_depth = 1).fit(train_imputed,train.loc[:,'Churn Category'])

[[122 185   0   0   0   0]
 [  0  53   0   0   0   0]
 [  0  20   0   0   0   0]
 [  0  19   0   0   0   0]
 [  0  13   0   0   0   0]
 [  0  10   0   0   0   0]]
0.14484130914725815


### knn 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 50).fit(x_train, y_train)
predictions = knn.predict(x_test)
matrix=confusion_matrix(y_test,predictions)
print(matrix)
score=f1_score(y_test,predictions,average='macro')
print(score)


[[302   5   0   0   0   0]
 [ 36  17   0   0   0   0]
 [ 14   6   0   0   0   0]
 [ 13   6   0   0   0   0]
 [  9   4   0   0   0   0]
 [  8   2   0   0   0   0]]
0.20703736650176094


### RandomForest + ada boost

In [ ]:
from sklearn.ensemble import RandomForestClassifier
for tree in range(10,11):
    for d in range(10,11):
        rf = RandomForestClassifier(n_estimators = tree*1000, max_depth=d,oob_score=True)
        rf.fit(x_train,y_train)
        print(f"tree:{tree*1000},d:{d},acc:{rf.oob_score_}")
        dtree_predictions = rf.predict(x_test)
        matrix=confusion_matrix(y_test,dtree_predictions)
        print(matrix)
        score=f1_score(y_test,dtree_predictions,average='macro')
        print(score)
rf.fit(train_imputed,train.loc[:,'Churn Category'])


KeyboardInterrupt: 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
est=60
for i in range(1):
    ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2),n_estimators = est,learning_rate=0.2)
    ada.fit(x_train, y_train)
    print("ada",est,ada.score(x_test,y_test))
    pre=ada.predict(x_test)
    score=f1_score(y_test,pre,average='macro')
    print("f1 score",score)
    matrix=confusion_matrix(y_test,pre)
    print(matrix)
    est+=15
x_all, y_all = ros.fit_resample(train_imputed, train.loc[:,'Churn Category'])
ada.fit(x_all,y_all)


ada 60 0.7180094786729858
f1 score 0.3632199276318573
[[258  36   2   4   0   7]
 [  2  32   9   3   2   5]
 [  2   8   4   4   0   2]
 [  1   6   3   4   0   5]
 [  3   8   0   0   2   0]
 [  0   4   1   1   1   3]]


AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2),
                   learning_rate=0.2, n_estimators=60)

### Prediction

### handle test

In [18]:
testID=pd.read_csv('Test_IDs.csv')
testID.columns=['Customer ID']
dftest=pd.DataFrame(testID)
test=pd.merge(dftest,users,on='Customer ID',how='left')
#print(test)
test_imputed = imputer.fit_transform(test.loc[:, features])
df=pd.DataFrame(test_imputed)
poly = PolynomialFeatures(degree=2, interaction_only=False)
test_imputed = pd.DataFrame(poly.fit_transform(df))
scalar=StandardScaler()
test_imputed=scalar.fit_transform(test_imputed)
test_imputed = pd.DataFrame(test_imputed)
#print(train_imputed.shape)

### SVM prediction

In [19]:
#dftest['Churn Category']=svm_model_linear.predict(test_imputed)
total_rows=test_imputed.shape[0]
test_imputed=test_imputed.to_numpy()
fake_y=np.zeros(total_rows)
p_label, p_acc, p_val=svm_predict(fake_y,test_imputed,libsvm_train)
p_label = list(map(int, p_label))
dftest['Churn Category']=p_label


Accuracy = 76.0823% (1072/1409) (classification)


### not svm

In [20]:
#dftest['Churn Category']=dtree_model.predict(test_imputed)

#dftest['Churn Category']=knn.predict(test_imputed)

#dftest['Churn Category']=rf.predict(test_imputed)

#dftest['Churn Category']=ada.predict(test_imputed)

### Output result

In [21]:
dftest.columns=['Customer ID','Churn Category']
submiss=pd.DataFrame(dftest)
submiss.to_csv('submission.csv',index=False)